# Lab 9
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - networking

In [108]:
!docker run -d --name nginx2 --network=none -p 81:81 nginx

438822a9b3b506f600624a19599791b7179704b8fafd87d265f0ecbbf21d9528


we now have an nginx container 

In [6]:
!curl localhost

curl: (7) Failed to connect to localhost port 80 after 0 ms: Couldn't connect to server


In [15]:
!docker exec -it nginx /bin/sh -c "curl google.com"



curl: (6) Could not resolve host: google.com


As you can see the nginx container can not talk to google.com

Docker comes with a subcommand for handling Docker networks. To get started, try the following command:

In [16]:
!docker network


Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


In [109]:
!docker network ls # see what networks are avalable

NETWORK ID     NAME       DRIVER    SCOPE
6d4b4774f6a9   bridge     bridge    local
665ff80aad7d   database   bridge    local
c5a5ffb553b3   host       host      local
8d9ad3b847c5   none       null      local


In [110]:
!docker network inspect none # note the nginx

[
    {
        "Name": "none",
        "Id": "8d9ad3b847c5be2d1a30063903cd890c9a9ac6fa794e58dcbae944e0884516f1",
        "Created": "2023-04-17T19:40:35.58322546Z",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "1e5671b581d1f1b693b2a76f5613922ab271bf48dc53e9681ecfa7934cb41fca": {
                "Name": "nginx",
                "EndpointID": "3a2d34fa7cb019d60190c107908773f5cf5f20d601151be15295ac459aa525ce",
                "MacAddress": "",
                "IPv4Address": "",
                "IPv6Address": ""
            },
            "438822a9b3b506f600624a19599791b7179704b8fafd87d265f0ecbbf21d9528": {
                "Name": "n

In [111]:
#setup another conainer for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 
!docker logs -f mysql

bd7be19c924fd7887bcd3c018ea34a76673cf52475d684f25686314fff083791
2024-04-09 02:23:45+00:00 [Note] [Entrypoint]: Entrypoint script for MySQL Server 8.3.0-1.el8 started.
2024-04-09 02:23:45+00:00 [Note] [Entrypoint]: Switching to dedicated user 'mysql'
2024-04-09 02:23:45+00:00 [Note] [Entrypoint]: Entrypoint script for MySQL Server 8.3.0-1.el8 started.
2024-04-09 02:23:45+00:00 [Note] [Entrypoint]: Initializing database files
2024-04-09T02:23:45.623036Z 0 [System] [MY-015017] [Server] MySQL Server Initialization - start.
2024-04-09T02:23:45.624217Z 0 [System] [MY-013169] [Server] /usr/sbin/mysqld (mysqld 8.3.0) initializing of server in progress as process 80
2024-04-09T02:23:45.628192Z 1 [System] [MY-013576] [InnoDB] InnoDB initialization has started.
2024-04-09T02:23:45.847813Z 1 [System] [MY-013577] [InnoDB] InnoDB initialization has ended.
2024-04-09T02:23:46.434930Z 6 [Warning] [MY-010453] [Server] root@localhost is created with an empty password ! Please consider switching off the

## The following needs to be run in the terminal

In [113]:
!docker run -d --name adminer -p 8080:8080 adminer


7fa95a01b8afcbd30543c852388c9597fc460070001b9b2400959c408253ee8c


In [114]:
!docker logs -f adminer

[Tue Apr  9 02:25:33 2024] PHP 7.4.33 Development Server (http://[::]:8080) started
^C


In [115]:
!docker inspect mysql | grep -n1 "IPAddress\":"


212-            "GlobalIPv6PrefixLen": 0,
213:            "IPAddress": "172.17.0.2",
214-            "IPPrefixLen": 16,
--
225-                    "Gateway": "172.17.0.1",
226:                    "IPAddress": "172.17.0.2",
227-                    "IPPrefixLen": 16,


In [116]:
!docker inspect adminer| grep  -n1 "IPAddress\":"

204-            "GlobalIPv6PrefixLen": 0,
205:            "IPAddress": "172.17.0.3",
206-            "IPPrefixLen": 16,
--
217-                    "Gateway": "172.17.0.1",
218:                    "IPAddress": "172.17.0.3",
219-                    "IPPrefixLen": 16,


## Creating Named so that other containers can see other containers not just through the host

In [141]:
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql

adminer
adminer
mysql
mysql


In [118]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

dd243e991b929b84fb5343502e6a974e5678cd2cd7427d5d745a99b5146aef9d


In [142]:
!docker network inspect yourNetwork

[
    {
        "Name": "yourNetwork",
        "Id": "dd243e991b929b84fb5343502e6a974e5678cd2cd7427d5d745a99b5146aef9d",
        "Created": "2024-04-09T02:40:43.611900004Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.19.0.0/16",
                    "Gateway": "172.19.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "13b05c4725ca287d98adf75ef9213c96c3fb49c5d5827ad605fecb95d8964ae0": {
                "Name": "mysql2",
                "EndpointID": "01f528debbab0845e01fc47d05d303f7dd067b3a89ca7ee3f641af83b8117a07",
                "MacAddress": "02:42:ac:13:00:02",
                "IPv4Address

In [149]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


Unable to find image 'mysql:8' locally
8: Pulling from library/mysql

17b3cebc: Pulling fs layer 
022566ed: Pulling fs layer 
18f4775b: Pulling fs layer 
fab8150e: Pulling fs layer 
1470e9ce: Pulling fs layer 
c48356cf: Pulling fs layer 
4d40ba87: Pulling fs layer 
27488bbe: Pulling fs layer 
304473a0: Pulling fs layer 
Digest: sha256:0f2e15fb8b47db2518b1428239ed3e3fe6a6693401b2cf19552063562cfc2fc4[2KExtracting  38.44MB/64.03MB
Status: Downloaded newer image for mysql:8
9b85b45e6cf73949b55855ac695d9049dedd002578e8ed8a97ef4845559071e6


In [150]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

Unable to find image 'adminer:latest' locally
latest: Pulling from library/adminer

e134f2f5: Pulling fs layer 
482dff10: Pulling fs layer 
285c8fa7: Pulling fs layer 
f7875cdf: Pulling fs layer 
b99054a5: Pulling fs layer 
d715c1ea: Pulling fs layer 
Digest: sha256:b75eae89431e8469613b844e76382a26efc8601c17f446bcd81665bc87ca9a1f
Status: Downloaded newer image for adminer:latest
6d712782ff62f841a79b5a360b42592dab66566d9e40125c1eda32e6e62785a4


In [151]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

Error response from daemon: endpoint with name adminer already exists in network yourNetwork


In [139]:
!docker inspect mysql2 |grep -C5 " \"Networks\""

            "GlobalIPv6PrefixLen": 0,
            "IPAddress": "",
            "IPPrefixLen": 0,
            "IPv6Gateway": "",
            "MacAddress": "",
            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,
                    "Links": null,
                    "Aliases": [
                        "13b05c4725ca"


In [148]:
!docker inspect adminer| grep  -C5 " \"Networks\""

            "GlobalIPv6PrefixLen": 0,
            "IPAddress": "",
            "IPPrefixLen": 0,
            "IPv6Gateway": "",
            "MacAddress": "",
            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,
                    "Links": null,
                    "Aliases": [
                        "7321bb69f22a"


# Part 2 
You will need to do a little digging here and work on your own!

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 


# Part 3
Continue with the above code and create a script that will relaunch a container if it ever shutsdown. please use the following web page to start your code and understand Docker API.

https://docs.docker.com/engine/api/sdk/examples/

